In [ ]:
# package import
import chipwhisperer as cw
from cw_plugins.scopes import *
from cw_plugins.targets import *
from tqdm import tqdm
import gui_helper as gui


### VISA Instrument Selector
Select a VISA instrument of an oscilloscope you want to use.

In [ ]:
display(gui.get_inst_sel())

### Connect to the selected oscilloscope

In [ ]:
# connect to the oscilloscope
scope = Oscilloscope(gui.get_visa_address())

### Configure channel settings
At least, you need to configure trigger settings and trace settings by the following methods.

`config_trigger_channel(mode, channel, scale, offset)`
* `mode`: Trigger mode.
  * TriggerMode.EDGE_RISE: trigger with rising edge.
  * TriggerMode.EDGE_FALL: trigger with falling edge.
  * TriggerMode.EDGE_ANY: trigger with both rising and falling edges.
* `channel`: channel number
* `scale`: vertical scale (V/div)
* `offset`: vertical offset (V)

`config_trace_channel(channel, scale, offset, period, delay = 0, ....)`
* `channel`: channel number
* `scale`: vertical scale (V/div)
* `offset`: vertical offset (V)
* `period`: acquired waveform period (s)
* `delay`: delay time to start acquiring waveform after trigger (s)
* `...`: other device-specific parameters


In [ ]:
# setting the scope
# channel 3 is the trigger channel
scope.config_trigger_channel(TriggerMode.EDGE_FALL, 3, 1, 0)
# channel 1 is the trace channel, 2mV/div, trace period 4us
scope.config_trace_channel(1, milliVolt(2), -0.99, microSecond(4), delay = 0, impedance=50)


### Target device
This plugin provides additional target devices as follows:
* `SakuraX`: Sakura-X board configured with its reference design.  
  - Arguments:
    * serial_port (str): path to the serial port connected to the Sakura-X board.
* `ESP32`: ESP32 board  
  - Arguments:
    * serial_port (str): path to the serial port connected to the ESP32 board.  
  - Options:  
    * baudrate (int): baudrate of the serial port.
* `SakuraXShellExampleAES128BitRTL`, `SakuraXShellExampleAES128BitHLS`  
Sample of AES128 implementation using Sakura-X Shell.  
  - Arguments:
    * serial_port (str): path to the serial port connected to the Sakura-X board.
* `SakuraXVexRISCVControlAES128bit`: Software implementation of AES running on VexRiscv_SakuraX
  - Arguments:
    * serial_port (str): path to the serial port connected to the Sakura-X board
  - Options:  
    * masked (bool): Use masked AES implmentation (Default: False)

In [ ]:
display(gui.get_comport_sel())

In [ ]:
# connect to the target
from cw_plugins.targets import *
target = cw.target(scope, SakuraX, serial_port = gui.get_comport())

### Create project
Create a project with the following code.

In [ ]:
### create chipwhisperer project
project = cw.create_project("tmp", overwrite=True)

### Trace capturing setup
`gui.get_waveform_pane` gives a waveform viewer.
The last argument specifies how often to update preview.
If you don't need the preview, comment out 

In [ ]:
# acquire traces
ktp = cw.ktp.Basic()
# generate a key
key = ktp.next_key()
print("key: ", key)

trace_count = 10
progress_bar = gui.TqdmWidget(max = trace_count, description="Capturing traces")
wave_pane = gui.get_waveform_pane(project, progress_bar, 2) 

### Start trace capturing

In [ ]:
progress_bar.display()
display(wave_pane)
while len(project.traces) < trace_count:
    text = ktp.next_text()
    trace = cw.capture_trace(scope, target, text, key, as_int = True)
    if trace is None:
        continue
    project.traces.append(trace)
    progress_bar.update(1)

### Save project
Save the project by the following code.

In [ ]:
# save the project
project.save()